### Install libraries

In [1]:
# pip install pyTelegramBotAPI

In [2]:
# pip install requests

In [3]:
# pip install io

### Configurations

In [4]:
BOT_TOKEN='BotFather Token'

In [5]:
import telebot
import requests
from io import BytesIO
import matplotlib.pyplot as plt

In [6]:
bot = telebot.TeleBot(BOT_TOKEN)

In [7]:
lookup_url = 'https://www.openfisheries.org/api/landings/countries.json'
endpoint_url = 'https://www.openfisheries.org/api/landings/countries/{iso3c}.json'

### Customize your bot

In [8]:
@bot.message_handler(commands=['start'])
def start(message):
    try:
        # Fetch the JSON data from the lookup endpoint
        response = requests.get(lookup_url)
        response.raise_for_status()
        countries_data = response.json()

        # Sort the countries by name
        sorted_countries = sorted(countries_data, key=lambda x: x['country'])

        # Create and send the lookup message as a document
        lookup_message = "Country Name: ISO3 Code\n"
        for country in sorted_countries:
            lookup_message += f"{country['country']}: {country['iso3c']}\n"

        # Send the message as a document
        document = BytesIO(lookup_message.encode())
        document.name = "Countries.txt"
        bot.send_document(message.chat.id, document, caption='Here is the lookup data as a document.')

        # Ask user for ISO3 code
        bot.send_message(message.chat.id, "Enter the ISO3 code of the country you want more details about:")

        # Register the next message from the user
        bot.register_next_step_handler(message, process_iso3c)

    except requests.exceptions.RequestException as e:
        bot.send_message(message.chat.id, f"Error fetching JSON data: {e}")

In [9]:
@bot.message_handler(func=lambda message: True)
def process_iso3c(message):
    try:
        iso3c = message.text.upper()  # Convert to uppercase
        endpoint = endpoint_url.format(iso3c=iso3c)

        # Fetch data using the provided endpoint
        response = requests.get(endpoint)
        response.raise_for_status()
        country_data = response.json()

        # Format the data into a neat and human-readable way
        formatted_data = "{:<12} {:<8}\n".format("Year", "Catch")
        for entry in country_data:
            rounded_catch = round(entry['catch'])
            formatted_data += "{:<12} {:<8}\n".format(entry['year'], rounded_catch)

        # Send the formatted data to the user
        bot.send_message(message.chat.id, formatted_data)

    except requests.exceptions.RequestException as e:
        bot.send_message(message.chat.id, f"Error fetching country data: {e}")

### Host bot

In [ ]:
bot.infinity_polling() # bot will be active only when running